# Verwendete Bibliotheken

In [1]:
import pandas as pd
import numpy as np

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPRegressor

import plotly.express as px
import plotly.graph_objects as go


# Einlesen der Daten

In [2]:
df = pd.read_csv("data/ticket_slot_analysis.csv")
df.head(10)

,tv_name,weekday,timeslot,number_of_tickets,Date
0,Luminor Arc 65,Friday,8-12,323,2025-11-28
1,StellarWave A9,Wednesday,8-12,2,2025-11-26
2,Luminor Arc 65,Monday,16-20,2,2025-11-24
3,StellarWave A9,Monday,0-8,1,2025-11-17
4,StellarWave A9,Tuesday,8-12,0,2025-11-11
5,StellarWave A9,Sunday,20-24,0,2025-11-09
6,Luminor Arc 65,Tuesday,8-12,274,2025-11-04
7,Luminor Arc 65,Sunday,0-8,0,2025-11-02
8,Luminor Arc 65,Monday,8-12,237,2025-10-27
9,StellarWave A9,Sunday,0-8,245,2025-10-26


# Visualisierung der Einflussfaktoren

In [15]:
# Scatterplot erstellen
fig = px.scatter(
    data_frame=df.groupby(['tv_name', 'weekday', 'timeslot'], as_index=False).agg(no_tickets=('number_of_tickets', 'sum')), 
    x="weekday", 
    y="timeslot",     
    color="tv_name", color_discrete_map={"StellarWave A9": "#08300d",   "Luminor Arc 65": "#ff7f0e" },
    category_orders=dict(weekday=["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"],
                        timeslot=["20-24", "16-20", "12-16", "8-12", "0-8"],),
    size="no_tickets",
    height=500,
    width=1200,
    title="Number of Tickets by TV Name, Weekday and Timeslot (Size indicates number of tickets)",
    )

fig.add_layout_image(dict(source="logo.png", x=1.02, y=1.26, sizex=0.15, sizey=0.15,))
fig.show()

# Trainieren eines neuronalen Netzes

In [3]:
X = df[["tv_name", "weekday", "timeslot"]]  # Features
y = df["number_of_tickets"]  # Target

hidden_layer_sizes = (8, )  # Hier können Sie die Anzahl Neuronen im Hidden Layer verändern

In [4]:
# Vorverarbeitung der kategorialen Merkmale
cat_cols = ["tv_name", "weekday", "timeslot"]
preprocess = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), cat_cols)
    ],
    remainder="drop"
)

# Anlegen eines einfachen Multi Layer Perceptron: 1 Hidden-Layer mit 8 Neuronen
mlp = MLPRegressor(
    hidden_layer_sizes=hidden_layer_sizes,
    activation="relu",
    solver="adam",
    random_state=42,
    max_iter=500
)

# Pipeline aus Vorverarbeitung und Modell erstellen
pipe = Pipeline(steps=[("prep", preprocess), ("mlp", mlp)])



In [5]:
# Auf historischen Daten trainieren
pipe = pipe.fit(X, y)

# Verwendung des trainierten Modells (Inferenz)

In [6]:
new_data = pd.DataFrame([["Luminor Arc 65", "Saturday", "0-8"], ["StellarWave A9", "Saturday", "0-8"]], columns=["tv_name", "weekday", "timeslot"])
new_data['prediction'] = pipe.predict(new_data[["tv_name", "weekday", "timeslot"]]).round(0).astype(int)
new_data

,tv_name,weekday,timeslot,prediction
0,Luminor Arc 65,Saturday,0-8,11
1,StellarWave A9,Saturday,0-8,267


# Visualisierung des neuronalen Netzwerks im Beispiel
Anzahl Tickets für Luminor Arc 65, Samstag, 0-8 Uhr

In [20]:
# Input der Features
x0 = pd.DataFrame([{
    "tv_name": "Luminor Arc 65",  
    # "tv_name": "StellarWave A9",
    "weekday": "Saturday",
    "timeslot": "0-8",
}])


In [21]:
# Interne Aufbereitung der Datenstrukturen


# Forward-Pass durch die Pipeline durchführen
pre = pipe.named_steps["prep"]
mlp = pipe.named_steps["mlp"]

# One-Hot-Transformation - Input-Vektor wird erzeugt
x_onehot = pre.transform(x0)
x_onehot = np.asarray(x_onehot).reshape(1, -1)

# Gewichte & Bias aus dem MLP extrahieren
W1, b1 = mlp.coefs_[0], mlp.intercepts_[0]      # (n_in, n_hidden), (n_hidden,)
W2, b2 = mlp.coefs_[1], mlp.intercepts_[1]      # (n_hidden, n_out), (n_out,)

# auf 1D-Output normalisieren:
if W2.ndim == 2 and W2.shape[1] == 1:
    W2 = W2[:, 0]                               # -> (n_hidden,)
if np.ndim(b2) > 0 and len(b2) == 1:
    b2 = b2[0]                                  # -> Skalar

# ReLU definieren
relu = lambda x: np.maximum(0, x)

# Hidden-Layer: Anwendung des Inputs im Hidden Layer
z1 = x_onehot @ W1 + b1  # Matrixmultiplikation: Input-Vektor * Gewichtsmatrix + Bias jedes Neurons
h1 = relu(z1)  # Aktivierung jedes Hidden-Neurons nach Anwendung von ReLU

# h1 ebenfalls 1D sichern
h1 = np.ravel(h1)                                # -> (n_hidden,)

# Output
y_vec = h1 @ W2 + b2
y_hat = np.ravel(y_vec)[0]  # Endgültige Vorhersage ermitteln durch Matrixmultiplikation des Hidden-Layer-Outputs mit den Output-Gewichten + Output-Bias
print(f"Vorhersage (Anzahl Tickets für {x0['tv_name'].values[0]}, {x0['weekday'].values[0]}, {x0['timeslot'].values[0]}): {y_hat:.1f}")

# aktive One-Hot-Indizes und Feature-Namen
feat_names = pre.get_feature_names_out()

x_onehot_vec = x_onehot.ravel()
active_idx = np.where(x_onehot_vec > 0.5)[0]


Vorhersage (Anzahl Tickets für Luminor Arc 65, Saturday, 0-8): 11.4


In [22]:
# Erstellung eines Sankey-Diagrams: Aktive Inputs → gefeuertes Hidden (h0…hN, oben→unten) → Output
# Dropdown "Input wählen", so dass nur die Pfade des gewählten Inputs farbig hervorhebt


# ---- Konfiguration Farben
petrol_scale = [
    "rgba(0, 48, 55, 0.9)", "rgba(0, 70, 80, 0.9)", "rgba(0, 90, 100, 0.9)",
    "rgba(0,110,120,0.9)", "rgba(0,130,140,0.9)", "rgba(0,150,160,0.9)",
    "rgba(0,170,180,0.9)", "rgba(0,190,200,0.9)",
]
POS_LINK = "rgba(0,160,0,0.78)"   # grün = positiver Beitrag
NEG_LINK = "rgba(220,0,0,0.78)"   # rot  = negativer Beitrag
EPS = 1e-6

def with_alpha(rgba: str, alpha: float) -> str:
    # "rgba(r, g, b, a)" -> gleiche Farbe mit neuem Alpha
    i = rgba.rfind(',')
    return rgba[:i] + f", {alpha})"

# ---- Daten aus Forward-Pass nutzen
active_inputs = [int(i) for i in active_idx]  # nur aktive One-Hots
h1 = np.ravel(h1)                             # (n_hidden,)
# NUR gefeuertes Hidden anzeigen (h>0)
fired_idx = [j for j, a in enumerate(h1) if a > EPS]
if len(fired_idx) == 0:
    # zur Not mindestens eins zeigen, damit kein leerer Plot entsteht
    fired_idx = [int(np.argmax(h1))]

# ---- Labels & Farben
input_labels  = [str(feat_names[i]) for i in active_inputs]
hidden_labels = [f"h{j}" for j in fired_idx]     # h0→hN (nur gefeuert)
output_labels = ["ŷ"]

input_colors  = [petrol_scale[i % len(petrol_scale)] for i in range(len(active_inputs))]
hidden_colors = [petrol_scale[j % len(petrol_scale)] for j in range(len(fired_idx))]
node_colors_base = input_colors + hidden_colors + ["rgba(200,200,200,0.65)"]  # Output grau

# ---- Pixel-bewusste Platzierung (verhindert Überlappungen)
THICK_PX = 16
PAD_PX   = 12
MARGIN_PX= 18
height_px = max(700, 100 + 28*(len(active_inputs) + len(fired_idx)))

def lane_y_positions(n, height_px, thick_px=THICK_PX, margin_px=MARGIN_PX):
    if n <= 1:
        return [0.5]
    unit = 1.0 / float(height_px)
    node_h   = thick_px * unit
    margin_u = margin_px * unit
    avail = 1.0 - 2*margin_u - n*node_h
    gap  = max(0.0, avail / (n-1))
    return [margin_u + i*(node_h + gap) for i in range(n)]

x_inputs  = [0.02] * len(active_inputs)
y_inputs  = lane_y_positions(len(active_inputs), height_px)

x_hidden  = [0.55] * len(fired_idx)
y_hidden  = lane_y_positions(len(fired_idx), height_px)  # Reihenfolge folgt fired_idx (h0 oben → ...)

x_output  = [0.98]
y_output  = [0.5]

node_x = x_inputs + x_hidden + x_output
node_y = y_inputs + y_hidden + y_output

# ---- Vollständige Label-Liste & Index-Offsets
labels            = input_labels + hidden_labels + output_labels
idx_input_start   = 0
idx_hidden_start  = len(input_labels)
idx_output        = idx_hidden_start + len(hidden_labels)

# ---- Links aufbauen (nur zu gefeuertem Hidden)
sources, targets, values, link_colors_base = [], [], [], []

# Input → Hidden: Gewicht W1[i_in, j_h]
for ii, i_in in enumerate(active_inputs):
    for jj, j_h in enumerate(fired_idx):
        w = float(W1[i_in, j_h])
        if abs(w) > EPS:
            sources.append(idx_input_start + ii)
            targets.append(idx_hidden_start + jj)
            values.append(abs(w))
            link_colors_base.append(POS_LINK if w >= 0 else NEG_LINK)

# Hidden → Output: Beitrag h1[j]*W2[j]
for jj, j_h in enumerate(fired_idx):
    c = float(h1[j_h] * W2[j_h])
    if abs(c) > EPS:
        sources.append(idx_hidden_start + jj)
        targets.append(idx_output)
        values.append(abs(c))
        link_colors_base.append(POS_LINK if c >= 0 else NEG_LINK)

# ---- Grunddiagramm
fig = go.Figure(data=[go.Sankey(
    arrangement="fixed",
    node=dict(
        pad=18, thickness=THICK_PX,
        label=labels, color=node_colors_base,
        x=node_x, y=node_y,
        line=dict(color="rgba(255,255,255,0.35)", width=0.6),
    ),
    link=dict(source=sources, target=targets, value=values, color=link_colors_base),
)])

# ---- Interaktive Dropdown-Buttons: "Alle Inputs" + je aktiver Input
buttons = []

# Helper: baue Farbarrays mit Highlight auf einen gewählten Input (index ii_sel)
def highlight_for_input(ii_sel):
    # --- 1) Verbundene Hidden-Knoten für den gewählten Input ermitteln
    connected_hidden_jj = set()
    for k, s in enumerate(sources):
        if s == idx_input_start + ii_sel:
            t = targets[k]
            if idx_hidden_start <= t < idx_output:
                connected_hidden_jj.add(t - idx_hidden_start)  # jj-Index in fired_idx

    # --- 2) Link-Farben: stark lassen wenn
    # (a) Input→Hidden-Kante vom selektierten Input ODER
    # (b) Hidden→Output-Kante von einem verbundenen Hidden
    link_cols = []
    for s, t, col in zip(sources, targets, link_colors_base):
        strong = False
        # (a) Input→Hidden vom selektierten Input
        if s == idx_input_start + ii_sel:
            strong = True
        # (b) Hidden→Output von verbundenem Hidden
        elif (idx_hidden_start <= s < idx_output) and (t == idx_output):
            jj = s - idx_hidden_start
            if jj in connected_hidden_jj:
                strong = True

        link_cols.append(col if strong else with_alpha(col, 0.08))

    # --- 3) Node-Farben: alles dimmen, selektierten Input + verbundene Hidden + Output hervorheben
    dim_inputs = [with_alpha(c, 0.25) for c in input_colors]
    dim_hidden = [with_alpha(c, 0.25) for c in hidden_colors]
    node_cols = dim_inputs + dim_hidden + ["rgba(200,200,200,0.4)"]

    # selektierter Input kräftig
    node_cols[ii_sel] = input_colors[ii_sel]
    # verbundene Hidden kräftig
    for jj in connected_hidden_jj:
        node_cols[idx_hidden_start + jj] = hidden_colors[jj]
    # Output etwas kräftiger
    node_cols[idx_output] = "rgba(160,160,160,0.85)"

    return link_cols, node_cols

# Button 0: alle Inputs (keine Dimmung)
buttons.append(dict(
    label="Alle Inputs",
    method="restyle",
    args=[{"link.color": [link_colors_base], "node.color": [node_colors_base]}],
))

# Buttons je Input
for ii, lbl in enumerate(input_labels):
    link_cols_hl, node_cols_hl = highlight_for_input(ii)
    buttons.append(dict(
        label=f"{lbl}",
        method="restyle",
        args=[{"link.color": [link_cols_hl], "node.color": [node_cols_hl]}],
    ))

fig.update_layout(
    title_text="Aktive Inputs → gefeuertes Hidden (h0…hN) → Output",
    font=dict(size=12),
    width=1150,
    height=height_px,
    updatemenus=[dict(
        type="dropdown",
        x=1.0, xanchor="right",
        y=1.12, yanchor="top",
        buttons=buttons,
        showactive=True
    )]
)

fig.show()



In [23]:
# Prognose ist Summe der Outputs der Hidden-Neuronen multipliziert mit den Output-Gewichten plus Bias
print(f"Bias des Output-Neurons: {mlp.intercepts_[1][0]:.2f}")  # Bias des Output-Neurons

#  grüner input -->  wegen dieses Features werden die jeweiligen Hidden Neuronen aktiv 
#  roter input -->  wegen dieses Features wird die Stärke eines Hideen Neurons verringert 



Bias des Output-Neurons: 2.00


# Heatmap gelernter Muster im neuronalen Netzwerk

In [24]:
# Visualisierung: Beiträge aller Neuronen im Hidden Layer abhängig von den Input-Features


M = np.abs(mlp.coefs_[0])  # shape = [n_features, n_hidden]
fnames = preprocess.get_feature_names_out(["tv_name","weekday","timeslot"])
neurons = [f"h{i}" for i in range(M.shape[1])]

df_heat = pd.DataFrame(M, index=fnames, columns=neurons).reset_index().melt(
    id_vars="index", var_name="hidden_neuron", value_name="weight_abs"
)
df_heat.rename(columns={"index": "input_feature"}, inplace=True)

# Plotly-Express-Heatmap mit transparenter Basis → Petrolverlauf
fig = px.imshow(
    M,  # Matrix mit Gewichtung der jeweiligen Neuronen
    x=neurons,
    y=fnames,
    color_continuous_scale=[
        (0.0, "rgba(255,255,255,0)"),  # transparent - kein Triggern
        (1.0, "#004F53")              # petrol - starkes Triggern
    ],
    aspect="auto",
    labels=dict(x="Hidden Neuron", y="Input Feature", color="Gewicht"),
    title="Gewichte der Neuronen in Abhängigkeit vom jeweiligen Input-Feature"
)

fig.update_layout(
    width=1000,
    height=700,
    xaxis=dict(side="top"),
    font=dict(size=10)
)

fig.show()
